In [ ]:
## Imports which will be necessary for dataframes(pandas) getting rid of NAs
## (numpy)

import pandas as pd
import numpy as np


In [ ]:
## This is how to connect your Colab notebook to your Google Drive files
## I have the

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Reading the CSV all_comments.csv which is the dataframe with has the
## following columns: date, title, text, subreddit, num of comments, url,
## full_comment (which are all of the comments of the post in one cell)


## Remember to change the path to your path
all_comments = pd.read_csv("/content/drive/MyDrive/DeepLearning/data/all_comments.csv", encoding='utf-8')

In [ ]:
## Having a look at the dataframe

all_comments.head()

Documentation for `pip install transformers`
https://pypi.org/project/transformers/
We put the `!` in front so we can write command line commands into the notebook

In [ ]:
## This is the command to install hugging face transformers intot the
## notebook, even if you have the package in your python environment
## you need to download it again for this notebook

!pip install transformers


In [ ]:
## Importing the pipeline for the model which we want to use
## see more instructions here
## https://huggingface.co/facebook/bart-large-mnli

from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [ ]:
## Replacing np.nan with "" so that I can create the full_text column
## which is title + text

all_comments = all_comments.replace(np.nan, "", regex=True)

In [ ]:
all_comments["full_text"] = all_comments['title'].str.cat(all_comments['text'], sep=' ')
all_comments.head()

In [ ]:
all_comments["full_text"].isna().sum()

In [ ]:
## This is the loop to get the label probabilities for the first five
## full_text observations

result = []
for i in range(5):
  sequence_to_classify = all_comments["full_text"][i]
  candidate_labels = ["justice", "fairness", "responsibility", "privacy", "transparency", "bias", "discrimination", "accountability"]
  result.append(classifier(sequence_to_classify, candidate_labels))

In [ ]:
## Looking at what the result looks like for the 3rd observation
## Indexing in python starts at 0, so result[2] is the third entry

result[2]

In [ ]:
## Checking if all the label probabilities are equal to 1

sum(result[2]["scores"])

In [ ]:
## Running for all of the full_text observations (takes for ever)

full_result = []
for i in range(len(all_comments)):
  print("-"*100)
  sequence_to_classify = all_comments["full_text"][i]
  candidate_labels = ["justice", "fairness", "responsibility", "privacy", "transparency", "bias", "discrimination", "accountability"]
  full_result.append(classifier(sequence_to_classify, candidate_labels))
  if i in range(0,367,10):
    print(f"Fetching data for {i}")

In [ ]:
full_result

In [ ]:
## Creating a dataframe from full_results
## Here I want one column where sequence = full_text
## And then there are 8 more columns for each of the label probabilities

df = pd.DataFrame([{
    'sequence': trial['sequence'],
    **{label: score for label, score in zip(trial['labels'], trial['scores'])}
} for trial in full_result])

In [ ]:
## So my loop above broke because my computer went to sleep oopsss
## at observation 339 out of 367 so I had to run the loop again from 340
## to 367

## Here with df.tail() I am checking where the loop stopped

df.tail()

In [ ]:
## I am saving the df up to entry 399 just for security, if the
## loop breaks again

df.to_csv("/content/drive/MyDrive/DeepLearning/data/df_339.csv")

In [ ]:
## Here I'm running again for the observations 340 to 367

last_result = []
for i in range(340, len(all_comments)+1):
  print("-"*100)
  sequence_to_classify = all_comments["full_text"][i]
  candidate_labels = ["justice", "fairness", "responsibility", "privacy", "transparency", "bias", "discrimination", "accountability"]
  last_result.append(classifier(sequence_to_classify, candidate_labels))
  if i in range(340,368,10):
    print(f"Fetching data for {i}")

In [ ]:
## Checking that last_result (observations 340 to 367) and full_result
## (observations 0 to 399) are equal to 367, that I have managed to run for
## all of them

len(last_result) + len(full_result)

In [ ]:
## Creating the df for observations 340 to 367 and saving it as a csv
## Remember to change the path if you want to save csv

df_399_367 = pd.DataFrame([{
    'sequence': trial['sequence'],
    **{label: score for label, score in zip(trial['labels'], trial['scores'])}
        } for trial in last_result])
df_399_367.to_csv("/content/drive/MyDrive/DeepLearning/data/df_339_367.csv")

In [ ]:
## Concatenating the 2 dfs

full_df = pd.concat([df, df_399_367])

In [ ]:
len(full_df)

In [ ]:
## Saving the full df

full_df.to_csv("/content/drive/MyDrive/DeepLearning/data/classification_all_posts.csv")

In [ ]:
all_comments.head()

In [ ]:
## Trying to run on the "full_comment" column but it never finished

comment_result = []
for i in range(len(all_comments)):
  print("-"*100)
  sequence_to_classify = all_comments["full_comment"][i]
  if sequence_to_classify != "":
    candidate_labels = ["justice", "fairness", "responsibility", "privacy", "transparency", "bias", "discrimination", "accountability"]
    comment_result.append(classifier(sequence_to_classify, candidate_labels))
  if i in range(0,367,10):
    print(f"Fetching data for {i}")

In [ ]:
## This is how you would save the comment label probabilities

comment_result = pd.DataFrame([{
    'sequence': trial['sequence'],
    **{label: score for label, score in zip(trial['labels'], trial['scores'])}
        } for trial in last_result])
comment_result.to_csv("/content/drive/MyDrive/DeepLearning/data/comment_result.csv")